In [8]:
API = '5854451716:AAF6h7NpNFjjb3OREkrryXsu41VhOBmZE5Q'

import telebot
import os
from telebot import types

import numpy as np
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import PIL
from PIL import Image
import math

import subprocess

In [9]:
processes = {}

In [10]:
bot = telebot.TeleBot(API)

In [11]:
start_reply = f'Hello. This bot converts your image and audio into a fancy spinning record circle video.\n\
To start, send messages with image and audio attachments'

In [12]:
Replies = {
    'start' : start_reply
}

In [13]:
@bot.message_handler(commands=['start'])
def start(message):
    bot.send_message(message.chat.id, Replies['start'])

@bot.message_handler(commands=['supertest'])
def supertest(message):
    with open('./final.mp4', 'rb') as vn:
        bot.send_video_note(message.from_user.id, data=vn, duration=60, length=int(1000/3))

@bot.message_handler(content_types=['audio'])
def receive_audio(message):

    if (message.from_user.id in processes.keys()):
        if (processes[message.from_user.id].poll() != 0):
            bot.send_message(message.chat.id, 'Wait until I finish the previous video!')
            return

    audio_file = bot.get_file(message.audio.file_id)
    downloaded_audio_file = bot.download_file(audio_file.file_path)

    path = f'./{message.from_user.id}'
    if (f'{message.from_user.id}' not in os.listdir('./')):
        os.mkdir(path)

    with open(path + '/audio.mp3', 'wb') as f:
        f.write(downloaded_audio_file)

    if ('image.png' in os.listdir(path)):

        inline_markup = types.InlineKeyboardMarkup()
        item = types.InlineKeyboardButton('Create', callback_data='create')
        inline_markup.add(item)
        bot.send_message(message.chat.id, 'Now press create to start magic!', reply_markup=inline_markup)

    else:
        bot.send_message(message.chat.id, 'Now send image!')

@bot.message_handler(content_types=['photo'])
def receive_image(message):

    if (message.from_user.id in processes.keys()):
        if (processes[message.from_user.id].poll() != 0):
            bot.send_message(message.chat.id, 'Wait until I finish the previous video!')
            return

    image_file = bot.get_file(message.photo[-1].file_id)
    downloaded_image_file = bot.download_file(image_file.file_path)

    path = f'./{message.from_user.id}'
    if (f'{message.from_user.id}' not in os.listdir('./')):
        os.mkdir(path)

    with open(path + '/image.png', 'wb') as f:
        f.write(downloaded_image_file)

    if ('audio.mp3' in os.listdir(path)):
        inline_markup = types.InlineKeyboardMarkup()
        item = types.InlineKeyboardButton('Create', callback_data='create')
        inline_markup.add(item)
        bot.send_message(message.chat.id, 'Now press create to start magic!', reply_markup=inline_markup)

    else:
        bot.send_message(message.chat.id, 'Now send audio!')

@bot.callback_query_handler(lambda call: True)
def create(call):
    if call.data == 'create':
        id = call.from_user.id
        path = f'./{id}'

        if (id in processes.keys()):
            if (processes[id].poll() != 0):
                bot.send_message(call.message.chat.id, 'Wait until I finish the previous video!')
                return

        if ('audio.mp3' in os.listdir(path)) and ('image.png' in os.listdir(path)):
            bot.send_message(call.message.chat.id, 'I start creating!')
            proc = subprocess.Popen(['python', 'converter.py', str(id)])
            processes[id] = proc
            
        else:
            print(os.listdir(path))
            bot.send_message(call.message.chat.id, 'I fucked up!')
    else:
        bot.send_message(call.message.chat.id, 'Fuck!')
        return

In [14]:
bot.polling(non_stop=True)